<a href="https://colab.research.google.com/github/UKJaagadhep/Data-science-and-machine-learning/blob/main/Chatbot%20using%20Blenderbot/Chatbot_using_Blenderbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
from datasets import load_dataset
from transformers import BlenderbotTokenizerFast, TFBlenderbotForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import create_optimizer
import pandas as pd
import tensorflow as tf

In [3]:
num_samples = 4
max_length = 512
max_new_tokens = 32

# **DATASET PREPARATION**

In [21]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d christianlillelund/joe-rogan-experience-1169-elon-musk
!unzip "/content/joe-rogan-experience-1169-elon-musk.zip" -d "/content/dataset/"

Dataset URL: https://www.kaggle.com/datasets/christianlillelund/joe-rogan-experience-1169-elon-musk
License(s): CC0-1.0
  0% 0.00/59.1k [00:00<?, ?B/s]
100% 59.1k/59.1k [00:00<00:00, 78.0MB/s]
Archive:  /content/joe-rogan-experience-1169-elon-musk.zip
  inflating: /content/dataset/joe-rogan-experience-1169-elon-musk.csv  


In [22]:
filepath = "/content/dataset/joe-rogan-experience-1169-elon-musk.csv"
dataset = load_dataset("csv", data_files = filepath)

Generating train split: 0 examples [00:00, ? examples/s]

In [60]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Timestamp', 'Speaker', 'Text'],
        num_rows: 1831
    })
})

In [61]:
dataset['train'][0]

{'Timestamp': '[00:00:00]',
 'Speaker': 'Joe Rogan',
 'Text': 'Ah, ha, ha, ha. Four, three, two, one, boom. Thank you. Thanks for doing this, man. Really appreciate it.'}

In [4]:
model_id = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizerFast.from_pretrained(model_id,
                                                    truncation_side = "left"
                                                    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

In [23]:
data_array = []

tokenizer.pad_token = tokenizer.eos_token

In [24]:
for i in range(num_samples, len(dataset['train'])):
  discussion = ""
  bot_output = tokenizer.bos_token + dataset['train'][i]['Text'] + tokenizer.eos_token
  for j in reversed(range(i - num_samples, i)):
    discussion = tokenizer.bos_token + dataset['train'][j]['Text'] + tokenizer.eos_token + discussion
    data_array.append([discussion, bot_output])

In [25]:
pd.DataFrame(data_array, columns = ['discussion', 'bot_output']).to_csv('discussion.csv')

In [26]:
df = pd.read_csv('/content/discussion.csv')
df

,Unnamed: 0,discussion,bot_output
0,0,<s>Nice to meet you too.</s>,<s>And thanks for not lighting this place on f...
1,1,<s>It's very good to meet you.</s><s>Nice to m...,<s>And thanks for not lighting this place on f...
2,2,<s>You're welcome.</s><s>It's very good to mee...,<s>And thanks for not lighting this place on f...
3,3,"<s>Ah, ha, ha, ha. Four, three, two, one, boom...",<s>And thanks for not lighting this place on f...
4,4,<s>And thanks for not lighting this place on f...,<s>You're welcome. That's coming later.</s>
...,...,...,...
7303,7303,<s>It's true.</s><s>I believe it's true too. S...,"<s>All right, thank you.</s>"
7304,7304,"<s>All right, thank you.</s>","<s>Good night, everybody. END OF TRANSCRIPTAut..."
7305,7305,"<s>All you assholes out there, be nice. Be nic...","<s>Good night, everybody. END OF TRANSCRIPTAut..."
7306,7306,<s>You're welcome.</s><s>All you assholes out ...,"<s>Good night, everybody. END OF TRANSCRIPTAut..."


In [5]:
filepath = "/content/discussion.csv"
dataset = load_dataset('csv', data_files = filepath)

Generating train split: 0 examples [00:00, ? examples/s]

In [68]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'discussion', 'bot_output'],
        num_rows: 14584
    })
})

In [69]:
dataset['train'][0 : 3]

{'Unnamed: 0': [0, 1, 2],
 'discussion': ["<s>Well, the flame, we didn't put a lot of time into the flamethrower. This was an off-the-cuff thing. It's sort of a hobbycompany called the Boring Company, which started out as a joke, and we decided to make a real, and dig a tunnelunder LA. And then, other people asked us to dig tunnels. And so, we said yes in a few cases.</s>",
  "<s>How does one, just in the middle of doing all the things you do, create cars, rockets, all the stuff you're doing,constantly innovating, decide to just make a flamethrower? Where do you have the time for that?</s><s>Well, the flame, we didn't put a lot of time into the flamethrower. This was an off-the-cuff thing. It's sort of a hobbycompany called the Boring Company, which started out as a joke, and we decided to make a real, and dig a tunnelunder LA. And then, other people asked us to dig tunnels. And so, we said yes in a few cases.</s>",
  "<s>You're welcome. That's coming later.</s><s>How does one, just in

In [6]:
def preprocess_function(example):
  return tokenizer(
    example['discussion'],
    text_target = example['bot_output'],
    padding = 'max_length',
    max_length = max_length,
    truncation = True)

In [7]:
tokenized_dataset = dataset.map(
    preprocess_function, remove_columns = dataset["train"].column_names
)

Map:   0%|          | 0/7308 [00:00<?, ? examples/s]

In [72]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14584
    })
})

In [8]:
model = TFBlenderbotForConditionalGeneration.from_pretrained(model_id)

tf_model.h5:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBlenderbotForConditionalGeneration.

Some layers of TFBlenderbotForConditionalGeneration were not initialized from the model checkpoint at facebook/blenderbot-400M-distill and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer,
                                       model = model,
                                       #return_tensors = "tf"
                                       )

In [10]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    shuffle = True,
    batch_size = 4,
    collate_fn=data_collator
)

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [31]:
for batch in tf_train_dataset.take(1):
    print(type(batch['input_ids']))
    print(type(batch['attention_mask']))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [77]:
for i in tf_train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[   1,  281,  623, ...,    2,    2,    2],
       [   1,  678,  315, ...,    2,    2,    2],
       [   1, 4041,  312, ...,    2,    2,    2],
       [   1, 1167,   19, ...,    2,    2,    2]])>, 'attention_mask': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'labels': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[   1, 1445,   21, ...,    2,    2,    2],
       [   1, 4129,  392, ...,    2,    2,    2],
       [   1,  714,  315, ...,    2,    2,    2],
       [   1, 1145,  271, ...,    2,    2,    2]])>}


In [78]:
tf_train_dataset

<_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}>

In [11]:
def replacements(a): #to ignore bos_token and eos_token (Also the pad_token) by replacing them with -100
  for i in [1, 2]:
    condition = tf.equal(a, i)
    case_true = -100 * tf.ones_like(a)

    case_false = a
    a = tf.where(condition, case_true, case_false)
  return a

In [80]:
a = tf.constant([[1, 3, 234, 445, 2, 2, 2],
               [1, 3445, 234, 34, 23, 2, 2]])
replacements(a)

<tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[-100,    3,  234,  445, -100, -100, -100],
       [-100, 3445,  234,   34,   23, -100, -100]], dtype=int32)>

In [12]:
def prepare_labels(inputs):
  return {'input_ids' : inputs['input_ids'],
          'attention_mask' : inputs['attention_mask'],
          'labels' : replacements(inputs['labels'])}

In [13]:
train_dataset = tf_train_dataset.map(prepare_labels)

In [83]:
for i in train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[   1,  452,  341, ...,    2,    2,    2],
       [   1,  452,  341, ...,    2,    2,    2],
       [   1, 1167,   21, ...,    2,    2,    2],
       [   1, 1262,   19, ...,    2,    2,    2]])>, 'attention_mask': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'labels': <tf.Tensor: shape=(4, 512), dtype=int64, numpy=
array([[-100, 4870, 2240, ..., -100, -100, -100],
       [-100,  452,  341, ..., -100, -100, -100],
       [-100,  472, 5108, ..., -100, -100, -100],
       [-100,  976,  505, ..., -100, -100, -100]])>}


# **MODELING**

In [84]:
model.summary()

Model: "tf_blenderbot_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBlenderbotMainLay  multiple                  364802560 
 er)                                                             
                                                                 
 final_logits_bias (BiasLay  multiple                  8008      
 er)                                                             
                                                                 
Total params: 364810568 (1.36 GB)
Trainable params: 364802560 (1.36 GB)
Non-trainable params: 8008 (31.28 KB)
_________________________________________________________________


In [14]:
num_epochs = 2
num_train_steps = len(train_dataset) * num_epochs
optimizer, schedule = create_optimizer(
  init_lr = 6e-5,
  num_warmup_steps = 1_000,
  num_train_steps = num_train_steps,
)

In [34]:
num_train_steps

3654

In [15]:
model.compile(optimizer = optimizer)

In [16]:
history = model.fit(train_dataset, epochs = num_epochs)

Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1827/1827 [==============================] - 3374s 2s/step - loss: 2.6020
Epoch 2/2
1827/1827 [==============================] - 3237s 2s/step - loss: 0.6711


In [17]:
model.save_weights('/content/blenderbot.h5')

# **TESTING**

In [18]:
input_text = tokenizer.bos_token + "Hello Elon glad to have you on my podcast." + tokenizer.eos_token + tokenizer.bos_token + "Thanks for Having me." + tokenizer.eos_token + tokenizer.bos_token + "i heard you are building robots. Tell me more about them." + tokenizer.eos_token + tokenizer.bos_token + "Well... Currently working on a robot which can do all house chores for you " + tokenizer.eos_token + tokenizer.bos_token + "Can this robot be used in Mars?" + tokenizer.eos_token

In [19]:
print(input_text)

<s>Hello Elon glad to have you on my podcast.</s><s>Thanks for Having me.</s><s>i heard you are building robots. Tell me more about them.</s><s>Well... Currently working on a robot which can do all house chores for you </s><s>Can this robot be used in Mars?</s>


In [20]:
history = tokenizer(input_text, return_tensors = "tf")

In [ ]:
output = model.generate(**history, max_new_tokens = max_new_tokens, do_sample = True, top_p = 0.9, min_length = 10)
print(tokenizer.decode(output[0]))

In [ ]:
output_temp = model.generate(**history, max_new_tokens = max_new_tokens, do_sample = True, temperature = 1.0, top_k = 0, min_length = 10)
print(tokenizer.decode(output_temp[0]))

In [ ]:
output_temp_2 = model.generate(**history, max_new_tokens = max_new_tokens, do_sample = True, temperature = 2.0, top_k = 0, min_length = 10)
print(tokenizer.decode(output_temp_2[0]))

In [ ]:
output_temp_half = model.generate(**history, max_new_tokens = max_new_tokens, do_sample = True, temperature = 0.5, top_k = 0, min_length = 10)
print(tokenizer.decode(output_temp_half[0]))

In [ ]:
output_topk = model.generate(**history, max_new_tokens = max_new_tokens, do_sample = True, top_k = 50, min_length = 10)
print(tokenizer.decode(output_topk[0]))

In [ ]:
output_topk_temp = model.generate(**history, max_new_tokens = max_new_tokens, do_sample = True, temperature = 2.0, top_k = 50, min_length = 10)
print(tokenizer.decode(output_topk_temp[0]))

# **CHATBOT**

**A discussion between myself and Elon Musk**

In [ ]:
max_length = 1024
chat_input = ""

for step in range(10):
  my_text = input(">> Host : ")
  new_user_input_ids = tokenizer.encode(
      tokenizer.bos_token + my_text + tokenizer.eos_token, return_tensors = 'tf')
  if step > 0:
    #everytime we tokenize a dialogue we tokenize it with all the dialogues before it
    chat_input = chat_input + tokenizer.bos_token + chat_history + tokenizer.eos_token + tokenizer.bos_token + my_text + tokenizer.eos_token
    bot_input_ids = tokenizer.encode(chat_input, return_tensors = 'tf')

  else:
    chat_input = tokenizer.bos_token + my_text + tokenizer.eos_token
    bot_input_ids = tokenizer.encode(chat_input, return_tensors = 'tf')

  chat_history_ids = model.generate(
      bot_input_ids,
      max_length = max_length,
      do_sample = True,
      temperature = 2.0,
      top_k = 50,
      min_length = 10,
      '''max_new_tokens = 32,
      temperature = 0.5,
      top_k = 0'''
      )

  chat_history = tokenizer.decode(chat_history_ids[0], skip_special_tokens = True)

  print(">> Elon Musk: {}".format(tokenizer.decode(chat_history_ids[0], skip_special_tokens = True)))